## モジュールのimport

In [ ]:
# Module
import sys
sys.path.append("../")
from mod.numpyro_utility import *

# DataFrame, Numerical computation
import polars as pl
pl.Config(fmt_str_lengths = 100, tbl_cols = 100, tbl_rows = 100)
import pandas as pd
import numpy as np
import jax
import jax.numpy as jnp

# ベイズ推定
import numpyro
import numpyro.distributions as dist # 確率分布

# plot
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import arviz as az

# plotの設定
import json
def to_rc_dict(dict):
    """
    jsonファイルのdictを読み込む
    """
    return {f'{k1}.{k2}': v for k1,d in dict.items() for k2,v in d.items()}

file_path = "../mod/rcParams.json"
with open(file_path) as f: 
    plt.rcParams.update(to_rc_dict(json.load(f)))

# 日本語 or 英語の2択
import japanize_matplotlib
#plt.rcParams['font.family'] = "Times New Roman"

## 5.3 階層ベイズモデル

### 5.3.1 問題設定
3種類の花から3つずつデータを抽出して、がく片の長さと幅の1次関数近似を行う。

### 5.3.2 データ準備

In [ ]:
# データセットを読み込む
df = sns.load_dataset("iris")

# setosa を抽出する
df_setosa = df.query('species == "setosa"')
df_versicolor = df.query('species == "versicolor"')
df_virginica = df.query('species == "virginica"')

# 乱数により3個のインデックスを生成
import random
random.seed(42)
indexes =range(len(df_setosa))
sample_indexes=random.sample(indexes, 3)

# df0, df1, df2のデータ数をそれぞれ3行にする
df_setosa_sel = df_setosa.iloc[sample_indexes]
df_versicolor_sel = df_versicolor.iloc[sample_indexes]
df_virginica_sel = df_virginica.iloc[sample_indexes]

#　全部連結して一つにする
df_sel = pd.concat([df_setosa_sel, df_versicolor_sel, df_virginica_sel]).reset_index(drop=True)

# 加工結果の確認
display(df_sel)

In [ ]:
sns.scatterplot(
    x='sepal_length', y='sepal_width', hue='species', style='species',
    data=df_sel, s=100)
plt.title('抽出した計9個の観測値の散布図');

In [ ]:
X = jnp.array(df_sel['sepal_length'].values, dtype = float)
Y = jnp.array(df_sel['sepal_width'].values, dtype = float)
species = df_sel['species']
cl = jnp.array(pd.Categorical(species).codes, dtype = int)

# 結果確認
print(X)
print(Y)
print(species.values)
print(cl)

### 5.3.3 確率モデル定義
数式で整理して、プログラミングで実装する。\
前節よりもさらに複雑である。

1. がく片の長さ sepal_length $x$ とがく片の幅 sepal_width $y$ とを1次関数で近似できる。
    * $y_{i} \approx \omega_{0}[cl] + \omega_{1}[cl] x_{i}$
    * $cl$ は花の種類の通し番号
1. 1次関数の右辺を $\mu_{i}$ とおく。
    * $\mu_{i} \equiv \omega_{0}[cl] + \omega_{1}[cl] x_{i}$
1. がく片の幅 sepal_width $y$ が正規分布に従うと仮定する。
    * 形状パラメータの $\mu_{i}$ が説明変数によって変化すると仮定する。
    * $y_{i} \sim N(\mu_{i}, \epsilon^2)$
    * 正規分布の標準偏差 $\epsilon$ の情報はないので広めの値を取る
        * $\epsilon \sim HN(10^2)$
1. 切片や係数の値は花の種類ごとに異なる
    * $\omega_{0}[cl] \sim N(\mu_{0}[cl], \sigma_{0}[cl]^2)$
        * $\mu_{0}[cl] \sim N(0, 10^2)$
        * $\sigma_{0}[cl] \sim HN(10^2)$
    * $\omega_{1}[cl] \sim N(\mu[cl], \sigma[cl]^2)$
        * $\mu_{1}[cl] \sim N(0, 10^2)$
        * $\sigma_{1}[cl] \sim HN(10^2)$

In [ ]:
def model_hierarchical_bayes(X, Y = None, cl = None, n_groups = None):
    '''
        5.3節の3種類の花の1次関数近似の階層ベイズモデル
    '''
    # 4.1 $\omega_{0}[cl] \sim N(\mu_{0}[cl], \sigma_{0}[cl]^2)$
    μ_ω0 = numpyro.sample("μ_ω0", dist.Normal(loc = 0, scale = 10))
    σ_ω0 = numpyro.sample("σ_ω0", dist.HalfNormal(scale = 10))
    with numpyro.plate("group", n_groups):
        ω0 = numpyro.sample("ω0", dist.Normal(loc = μ_ω0, scale = σ_ω0))
    # $\omega_{1}[cl] \sim N(\mu[cl], \sigma[cl]^2)$
    μ_ω1 = numpyro.sample("μ_ω1", dist.Normal(loc = 0, scale = 10))
    σ_ω1 = numpyro.sample("σ_ω1", dist.HalfNormal(scale = 10))
    with numpyro.plate("group", n_groups):
        ω1 = numpyro.sample("ω1", dist.Normal(loc = μ_ω1, scale = σ_ω1))
    # 2. 1次関数の右辺を $\mu_{i}$ とおく。
    μ = numpyro.deterministic("μ", ω0[cl] + ω1[cl] * X)
    # 3. 正規分布の標準偏差は $\epsilon \sim HN(10^2)$ と仮定する。
    ε = numpyro.sample("ε", dist.HalfNormal(scale = 10))
    # ベクトル化
    N = len(X)
    with numpyro.plate("N", N):
        # 1. $y_{i} \sim N(\mu, \sigma^2)$
        numpyro.sample("Y", dist.Normal(loc = μ, scale = ε), obs = Y)

In [ ]:
model_args = {
    "X": X,
    "Y": Y,
    "cl": cl,
    "n_groups": len(species.unique())
}
try_render_model(model_hierarchical_bayes, render_name = "階層ベイズ", **model_args)

### 5.3.4 サンプリングと結果分析

In [ ]:
model_args = {
    "X": X,
    "Y": Y,
    "cl": cl,
    "n_groups": len(species.unique())
}
idata = run_mcmc(
    model_hierarchical_bayes,
    num_chains = 4,
    num_warmup = 1000,
    num_samples = 1000,
    thinning = 1,
    seed = 42,
    target_accept_prob = 0.8,
    log_likelihood = False,
    **model_args
)

In [ ]:
az.plot_trace(idata, compact = False, var_names = ["ω0", "ω1", "ε"])
plt.tight_layout()

In [ ]:
summary = az.summary(idata, var_names = ["ω0", "ω1", "ε"])
display(summary)

### 5.3.5 散布図と回帰直線の重ね描き

In [ ]:
# 元書籍とほぼ同じ
# alphaとbetaの平均値の導出
means = summary['mean']
ω0_0 = means['ω0[0]']
ω0_1 = means['ω0[1]']
ω0_2 = means['ω0[2]']
ω1_0 = means['ω1[0]']
ω1_1 = means['ω1[1]']
ω1_2 = means['ω1[2]']

# 回帰直線用座標値の計算
x_range = np.array([X.min()-0.1,X.max()+0.1])
y0_range = ω1_0 * x_range + ω0_0
y1_range = ω1_1 * x_range + ω0_1
y2_range = ω1_2 * x_range + ω0_2

# 散布図表示
sns.scatterplot(
    x='sepal_length', y='sepal_width', hue='species', style='species',
    data=df_sel, s=100)
plt.plot(x_range, y0_range, label='setosa')
plt.plot(x_range, y1_range, label='versicolor')
plt.plot(x_range, y2_range, label='virginica')
plt.legend();

In [ ]:
# 回帰直線の座標値計算
x_range = np.array([
    df['sepal_length'].min()-0.1,
    df['sepal_length'].max()+0.1])
y0_range = ω1_0 * x_range + ω0_0
y1_range = ω1_1 * x_range + ω0_1
y2_range = ω1_2 * x_range + ω0_2

# 散布図表示
sns.scatterplot(
    x='sepal_length', y='sepal_width', hue='species', style='species',
    s=50, data=df)
plt.plot(x_range, y0_range, label='setosa')
plt.plot(x_range, y1_range, label='versicolor')
plt.plot(x_range, y2_range, label='virginica')
plt.legend();